# possible bug: core execution does not pause when dispatcher FIFO is full

Each tProc v2 dispatcher output (waveform, data, or trigger output) has a FIFO, which by default is 512 words deep (`FIFO_DEPTH=9` https://github.com/awsch/qick-spin/blob/a76ab8dc0f4c3975af3ce6416b3a1b4d5bc77fbf/firmware/ip/qick_processor/src/axis_qick_processor.sv#L33). 
These FIFOs are filled as the program is executed, and empty as each pulse's scheduled time is reached.
A program that schedules a lot of pulses can fill up one of the FIFOs faster than the pulses get played.

The tProc is supposed to pause execution when any of the FIFOs is full. This was the behavior in tProc v1; in v2 there is a config flag that is supposed to control this behavior (but the default is to pause).
The manual describes this as "D_FIFO: Disable FIFO_FULL_PAUSE (default 0: used for Debug purpose)" - it says it's bit 10, but it's bit 11: https://github.com/awsch/qick-spin/blob/a76ab8dc0f4c3975af3ce6416b3a1b4d5bc77fbf/firmware/ip/qick_processor/src/qick_processor.sv#L215.

Anyway, it appears that this behavior doesn't actually work; the tProc does not pause-on-full, so fast sequences with more than 512 pulses on a single generator may lose the later pulses.

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
from qick import *
import matplotlib.pyplot as plt
from qick.asm_v2 import AveragerProgramV2
from qick.asm_v2 import QickSweep1D

In [ ]:
# soc = QickSoc('/data/fw/2024-09-29_111_tprocv2r21_standard/qick_111.bit')
# soc = QickSoc('/home/xilinx/jupyter_notebooks/fw/qick_tprocv2_111_standard_repo/qick_111.bit')
# soc = QickSoc('/home/xilinx/jupyter_notebooks/fw/qick_tprocv2_111_standard_1ch/qick_111.bit')
soc = QickSoc('/home/xilinx/jupyter_notebooks/fw/qick_tprocv2_111_standard_1ch_tproc_debug/qick_111.bit')
# soc = QickSoc('/home/xilinx/jupyter_notebooks/fw/qick_tprocv2_111_standard_1ch_tproc_debug_2/qick_111.bit')
soccfg = soc

print(soccfg)

In [ ]:
# Start Debug Bridge Server if present

try:
    bridge = soc._get_block('debug_bridge_0')
    # bridge_bak = getattr(soc,'debug_wrapper/debug_bridge_0') # another way to access the object
    # print(bridge)
    # bridge?

    bridge.start_xvc_server(bufferLen=4096, serverAddress="0.0.0.0", serverPort=2542, reconnect=True, verbose=True)

except:
    print('Debug blocks not found')


In [ ]:
hardware = {
        "drive_ch": 1,
        "ro_adc_ch": 0
    }

## example
This example plays 1000 short pulses and reads them out using the DDR4 buffer. The last two lines in the body slow down tProc execution to avoid FIFO overflow. If you comment them out, you will see that only half of the pulses get played.

Later on there is a test of setting the FIFO_FULL_PAUSE flag. This should disable pause-on-full, but it really seems that pause-on-full does not work regardless of the value of the flag.

In [ ]:
class ManyPulseProg(AveragerProgramV2):
    def _initialize(self, cfg):  
        ro_ch = cfg["ro_adc_ch"]
        gen_ch = cfg["drive_ch"]

        self.declare_gen(ch=gen_ch, nqz=cfg['nqz_drive'])
        self.declare_readout(ch=ro_ch, length=cfg['readout_length'])
        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['frequency_drive'], gen_ch=gen_ch)
        self.add_pulse(ch=gen_ch, name="pulse10", ro_ch=ro_ch, 
                   style="const", 
                   freq=cfg['frequency_drive'], 
                   length=cfg['len_drive'],
                   phase=0,
                   gain=1, 
                  )

        self.send_readoutconfig(ch=cfg['ro_adc_ch'], name="myro", t=0)
        self.delay(1.0) # create some execution slack for the trigger
        self.trigger(ddr4=True, t=0)
        
    def _body(self, cfg):
        for p in range(cfg['n_drive']):
            t = cfg['initial_drive'] + p*(cfg['len_drive']+cfg['gap_drive'])
            self.pulse(ch=cfg['drive_ch'], name = 'pulse10', t=t, tag="drivepulse%d"%(p))
            
            # if tProc execution gets too far ahead of the timeline, the pulse dispatcher will fill up
            # so: periodically insert a wait, to limit the execution slack to ~initial_drive
            # this is a dumb hack and you should not blindly reuse this (or use this ever, probably)
            # it's just a way to prove that we are seeing dispatcher overflow
            if cfg['wait_hack'] and p%100==0:
                self.wait(p*(cfg['len_drive']+cfg['gap_drive']))

In [ ]:
n = 1000
initial_start = 50
gap_between_pulses = 0.1
pulse_len = 0.1

drive = {
    "frequency_drive": 500,
    "nqz_drive":1,
    'initial_drive':initial_start,
    'len_drive':pulse_len,
    'gap_drive':gap_between_pulses,
    'n_drive':n,
}
expt_config = {
    "wait_hack": False,
    "readout_length":5, # this is not actually used, we don't use the regular buffer
}
config = {**hardware, **drive, **expt_config}

In [ ]:
config['wait_hack'] = False
soc.tproc.tproc_cfg = 0 # this is the default - just making sure
n_transfers = 2000
soc.arm_ddr4(ch=config['ro_adc_ch'], nt=n_transfers)
prog = ManyPulseProg(soccfg, reps=1, final_delay=10, cfg=config)

prog.run_rounds(soc, rounds=1)

# soc.tproc.print_debug()

iq_ddr4 = soc.get_ddr4(n_transfers)
t = prog.get_time_axis_ddr4(config['ro_adc_ch'], iq_ddr4)

fig = plt.figure()
plt.plot(t, iq_ddr4[:,0])
plt.plot(t, iq_ddr4[:,1])
plt.xlim([initial_start-5,initial_start + n*(pulse_len+gap_between_pulses)+10])
# if all the pulses play, they should mostly fill the plot range


In [ ]:
## Do not forget to stop the XVC server once you finish.
bridge.stop_xvc_server()


In [ ]:
print(prog)

In [ ]:
prog.print_pmem2hex()

In [ ]:
prog.print_wmem2hex()

In [ ]:
fig = plt.figure()
plt.plot(t, iq_ddr4[:,0])
plt.plot(t, iq_ddr4[:,1])
plt.xlim([initial_start-5,initial_start + n*(pulse_len+gap_between_pulses)+10])
# if all the pulses play, they should mostly fill the plot range

In [ ]:
# zooming in on the first few pulses, to check spacing
fig.axes[0].set_xlim([50,55])
fig

In [ ]:
# print as-compiled pulse times
for i in range(10):
    print(prog.get_time_param('drivepulse%d'%(i), 't'))

In [ ]:
config['wait_hack'] = True

n_transfers = 2000
soc.arm_ddr4(ch=config['ro_adc_ch'], nt=n_transfers)
prog = ManyPulseProg(soccfg, reps=1, final_delay=10, cfg=config)
iq_list = prog.run_rounds(soc, rounds=1)
iq_ddr4 = soc.get_ddr4(n_transfers)
t = prog.get_time_axis_ddr4(config['ro_adc_ch'], iq_ddr4)

fig = plt.figure()
plt.plot(t, iq_ddr4[:,0])
plt.plot(t, iq_ddr4[:,1])
plt.xlim([initial_start-5,initial_start + n*(pulse_len+gap_between_pulses)+10])

In [ ]:
config['wait_hack'] = False

soc.tproc.tproc_cfg = 1 << 11 # this should disable pause-on-full
print("setting tproc_cfg to", soc.tproc.tproc_cfg)

n_transfers = 2000
soc.arm_ddr4(ch=config['ro_adc_ch'], nt=n_transfers)
prog = ManyPulseProg(soccfg, reps=1, final_delay=10, cfg=config)
iq_list = prog.run_rounds(soc, rounds=1)
iq_ddr4 = soc.get_ddr4(n_transfers)
t = prog.get_time_axis_ddr4(config['ro_adc_ch'], iq_ddr4)

# print to confirm that tproc_cfg didn't get changed by the library
print("after run, tproc_cfg is", soc.tproc.tproc_cfg)

fig = plt.figure()
plt.plot(t, iq_ddr4[:,0])
plt.plot(t, iq_ddr4[:,1])
plt.xlim([initial_start-5,initial_start + n*(pulse_len+gap_between_pulses)+10])